In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]='11,13,14,15'

In [2]:
%matplotlib notebook
%matplotlib inline

OK_PATH='/Data_Storage/Rui_Data_Space/VQA/OK-VQA'
import pandas as pd
from collections import defaultdict

In [3]:
import json
import pickle as pkl

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [4]:
from PIL import Image
import os
import random

In [5]:
val_anno=load_json(os.path.join(OK_PATH,'mscoco_val2014_annotations.json'))['annotations']
ans_dict={}
for row in val_anno:
    ques_id=str(row['question_id'])
    answers=defaultdict(int)
    for info in row['answers']:
        answers[info['answer']]+=1
    #ans={ans:answers[ans]/sum(answers.values()) for ans in answers.keys()}
    ans_dict[ques_id]=answers
print (len(val_anno),len(ans_dict))

5046 5046


In [6]:
vqa_all=load_json(os.path.join(OK_PATH,'OpenEnded_mscoco_val2014_questions.json'))['questions']
print (len(vqa_all))

5046


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model = AutoModelForCausalLM.from_pretrained("facebook/opt-13b", 
                                             torch_dtype=torch.float16, 
                                             device_map="auto")

# the fast tokenizer currently does not work correctly

[2023-06-23 12:48:56,614] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-13b", 
                                          padding_side='left',
                                          use_fast=False)

In [9]:
syn_qa_pairs=load_pkl('../OK_VQA/ok_vqa_qa_img2llm.pkl')
print(len(syn_qa_pairs))

5046


In [10]:
def get_score(occurences):
    return min(1.0,occurences/3.0)

In [11]:
import inflect
single = inflect.engine()

def norm_ans(ans):
    if ans.endswith(',') or ans.endswith('.'):
        ans=ans[:-1]
    if ans.startswith('a '):
        ans=ans[2:]
    if ',' in ans:
        ans=ans.split(',')[0]
    if len(ans.split(' '))==1 and ans not in ['grass','glass','bus']:
        norm_ans=single.singular_noun(ans)
        if norm_ans !=False:
            ans=norm_ans
    return ans

In [16]:
NUM_DEMO=15
NUM_KB=5
NUM_CAP=50

head='Please answer the question accordint to the above context and knowledge with as few words as possible.\n'
def prompt_construction_kb(entry,ques,kb,contexts):
    questions=entry['questions']
    answers=entry['answers']
    captions=entry['captions']
    random.shuffle(captions)
    all_texts=[head+'\n']
    all_texts.append('Contexts:'+' . '.join(captions[:NUM_DEMO+NUM_KB])+'\n')
    all_texts.append('Knowledge:'+' '.join(kb[:NUM_KB])+'\n')
    for i,q in enumerate(questions[:NUM_DEMO]):
        all_texts.append('Question:'+q+'\nAnswer:'+answers[i])
    all_texts.append('Question:'+\
                     ques+\
                     '\nAnswer:')
    all_texts='\n'.join(all_texts)
    return all_texts

In [12]:
model.train(False)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 5120, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 5120)
      (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (v_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (out_proj): Linear(in_features=5120, out_features=5120, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=5120, out_features=20480, bias=True)
          (fc2): Linear(in_features=20480, out_features=5120, bias=True)
          (final_layer_norm): LayerNorm((5

In [13]:
#13b ablations
NUM_DEMO=15
NUM_KB=0
NUM_CAP=30

head='Please answer the question accordint to the above context and knowledge with as few words as possible.\n'
def prompt_construction_kb(entry,ques,kb,contexts):
    questions=entry['questions']
    answers=entry['answers']
    captions=entry['captions']
    random.shuffle(captions)
    all_texts=[head+'\n']
    all_texts.append('Contexts:'+' . '.join(captions[:NUM_DEMO])+'\n')
    #all_texts.append('Knowledge:'+' '.join(kb[:NUM_KB])+'\n')
    for i,q in enumerate(questions[:NUM_DEMO]):
        all_texts.append('Question:'+q+'\nAnswer:'+answers[i])
    all_texts.append('Question:'+\
                     ques+\
                     '\nAnswer:')
    all_texts='\n'.join(all_texts)
    return all_texts

#kb=0,num_cap=30,num_demo=15

vis=0
random.shuffle(vqa_all)
acc=0.0
for k,row in enumerate(vqa_all):
    if k>0 and k%200==0:
        print ('Already finished:',vis)
        print ('\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
    ques=row['question']
    answers=ans_dict[ques_id]
    entry=syn_qa_pairs[ques_id]
    kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',
                              ques_id+'.json'))
    captions=load_json('../OK_VQA/large_captions/'+ques_id+'.json')
    prompt=prompt_construction_kb(entry,ques,kb,captions)
    #print (prompt)
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt")
        output=model.generate(inputs['input_ids'].to(0), max_length=2000,
                              length_penalty=-1,
                              min_length=2,
                              eos_token_id=50118)
    pred=tokenizer.decode(output[0].tolist()).split(':')[-1].strip().lower()
    pred=norm_ans(pred)
    if pred in answers:
        acc+=get_score(answers[pred])
    #print('\t',ques,ans_dict[ques_id])
    #print (pred)
    vis+=1
print (acc*100.0/vis)

Already finished: 200
	Acc: 35.99999999999999
Already finished: 400
	Acc: 40.583333333333314
Already finished: 600
	Acc: 39.055555555555515
Already finished: 800
	Acc: 38.375
Already finished: 1000
	Acc: 37.63333333333338
Already finished: 1200
	Acc: 36.19444444444451
Already finished: 1400
	Acc: 36.071428571428655
Already finished: 1600
	Acc: 36.62500000000003
Already finished: 1800
	Acc: 37.07407407407406
Already finished: 2000
	Acc: 36.89999999999995
Already finished: 2200
	Acc: 37.19696969696961
Already finished: 2400
	Acc: 37.3472222222221
Already finished: 2600
	Acc: 37.34615384615371
Already finished: 2800
	Acc: 37.33333333333321
Already finished: 3000
	Acc: 37.17777777777771
Already finished: 3200
	Acc: 37.427083333333314
Already finished: 3400
	Acc: 37.13725490196082
Already finished: 3600
	Acc: 36.907407407407476
Already finished: 3800
	Acc: 37.24561403508784
Already finished: 4000
	Acc: 37.22500000000015
Already finished: 4200
	Acc: 37.38888888888907
Already finished: 4400
	

In [14]:
#13b ablations
NUM_DEMO=15
NUM_KB=5
NUM_CAP=30

head='Please answer the question accordint to the above context and knowledge with as few words as possible.\n'
def prompt_construction_kb(entry,ques,kb,contexts):
    questions=entry['questions']
    answers=entry['answers']
    captions=entry['captions']
    random.shuffle(captions)
    all_texts=[head+'\n']
    all_texts.append('Contexts:'+' . '.join(captions[:NUM_DEMO])+'\n')
    all_texts.append('Knowledge:'+' '.join(kb[:NUM_KB])+'\n')
    for i,q in enumerate(questions[:NUM_DEMO]):
        all_texts.append('Question:'+q+'\nAnswer:'+answers[i])
    all_texts.append('Question:'+\
                     ques+\
                     '\nAnswer:')
    all_texts='\n'.join(all_texts)
    return all_texts

#kb=0,num_cap=30,num_demo=15

vis=0
random.shuffle(vqa_all)
acc=0.0
for k,row in enumerate(vqa_all):
    if k>0 and k%200==0:
        print ('Already finished:',vis)
        print ('\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
    ques=row['question']
    answers=ans_dict[ques_id]
    entry=syn_qa_pairs[ques_id]
    kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',
                              ques_id+'.json'))
    captions=load_json('../OK_VQA/large_captions/'+ques_id+'.json')
    prompt=prompt_construction_kb(entry,ques,kb,captions)
    #print (prompt)
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt")
        output=model.generate(inputs['input_ids'].to(0), max_length=2000,
                              length_penalty=-1,
                              min_length=2,
                              eos_token_id=50118)
    pred=tokenizer.decode(output[0].tolist()).split(':')[-1].strip().lower()
    pred=norm_ans(pred)
    if pred in answers:
        acc+=get_score(answers[pred])
    #print('\t',ques,ans_dict[ques_id])
    #print (pred)
    vis+=1
print (acc*100.0/vis)

Already finished: 200
	Acc: 40.333333333333336
Already finished: 400
	Acc: 40.58333333333333
Already finished: 600
	Acc: 40.722222222222186
Already finished: 800
	Acc: 41.0
Already finished: 1000
	Acc: 41.200000000000024
Already finished: 1200
	Acc: 40.72222222222227
Already finished: 1400
	Acc: 39.80952380952384
Already finished: 1600
	Acc: 40.45833333333331
Already finished: 1800
	Acc: 40.57407407407402
Already finished: 2000
	Acc: 40.84999999999992
Already finished: 2200
	Acc: 40.83333333333322
Already finished: 2400
	Acc: 40.62499999999988
Already finished: 2600
	Acc: 40.51282051282042
Already finished: 2800
	Acc: 40.34523809523805
Already finished: 3000
	Acc: 40.5888888888889
Already finished: 3200
	Acc: 40.291666666666714
Already finished: 3400
	Acc: 40.12745098039225
Already finished: 3600
	Acc: 39.62037037037048
Already finished: 3800
	Acc: 39.710526315789615
Already finished: 4000
	Acc: 39.90833333333349
Already finished: 4200
	Acc: 40.23015873015891
Already finished: 4400
	Ac